# Day 7

In [2]:
import $ivy.`org.scalaz::scalaz-core:7.2.22`
import scalaz._
import Scalaz._

import $ivy.$                               

import scalaz._

import Scalaz._

## Applicative Builder

In [3]:
(3.some |@| 5.some) {_ + _}

res2: Option[Int] = Some(8)

In [38]:
val f = ({(_: Int) * 2} |@| {(_: Int) + 10}) {_ + _}

f: Int => Int = scalaz.std.FunctionInstances$$anon$3$$Lambda$3708/2043616620@477707c9

## 計算の状態の正体

In [4]:
type Stack = List[Int]

defined type Stack

In [5]:
def pop(stack: Stack): (Int, Stack) = stack match {
    case x :: xs => (x, xs)
}

defined function pop

In [6]:
def push(a: Int, stack: Stack): (Unit, Stack) = ((), a :: stack)

defined function push

In [7]:
def stackManip(stack: Stack): (Int, Stack) = {
    val (_, newStack1) = push(3, stack)
    val (a, newStack2) = pop(newStack1)
    pop(newStack2)
}

defined function stackManip

In [8]:
stackManip(List(5, 8, 2, 1))

res7: (Int, Stack) = (5, List(8, 2, 1))

## State and StateT

In [9]:
State[List[Int], Int] { case x :: xs => (xs, x) }

res8: State[List[Int], Int] = scalaz.IndexedStateT$$anon$10@7874d6e7

In [10]:
type Stack = List[Int]

defined type Stack

In [11]:
val pop = State[Stack, Int] {
    case x :: xs => (xs, x)
}

pop: State[Stack, Int] = scalaz.IndexedStateT$$anon$10@3c0a58a9

In [12]:
def push(a: Int) = State[Stack, Unit] {
    case xs => (a :: xs, ())
}

defined function push

In [13]:
def stackManip: State[Stack, Int] = for {
    _ <- push(3)
    a <- pop
    b <- pop
} yield(b)

defined function stackManip

In [14]:
stackManip(List(5, 8, 2, 1))

res13: (Stack, Int) = (List(8, 2, 1), 5)

## 状態の取得と設定

In [15]:
def stackyStack: State[Stack, Unit] = for {
    stackNow <- get
    r <- if (stackNow === List(1, 2, 3)) put(List(8, 3, 1))
        else put(List(9, 2, 1))
} yield r

defined function stackyStack

In [16]:
stackyStack(List(1, 2, 3))

res15: (Stack, Unit) = (List(8, 3, 1), ())

In [17]:
val pop: State[Stack, Int] = for {
    s <- get[Stack]
    val (x :: xs) = s
    _ <- put(xs)
} yield x

pop: State[Stack, Int] = scalaz.IndexedStateT$$anon$11@20937999

In [18]:
def push(x: Int): State[Stack, Unit] = for {
    xs <- get[Stack]
    r <- put(x :: xs)
} yield r

defined function push

## \/

In [19]:
1.right[String]

res18: String \/ Int = \/-(1)

In [20]:
"error".left[Int]

res19: String \/ Int = -\/("error")

In [21]:
Left[String, Int]("boom") flatMap { x => Right[String, Int](x + 1) }

res20: Either[String, Int] = Left("boom")

In [22]:
Left[String, Int]("boom").right flatMap { x => Right[String, Int](x + 1)}

res21: Either[String, Int] = Left("boom")

In [23]:
"boom".left[Int] >>= { x => (x + 1).right }

res22: String \/ Int = -\/("boom")

In [24]:
for {
    e1 <- "event 1 ok".right
    e2 <- "event 2 failed!".left[String]
    e3 <- "event 3 failed!".left[String]
} yield (e1 |+| e2 |+| e3)

res23: String \/ String = -\/("event 2 failed!")

In [25]:
"event 1 ok".right.isRight

res24: Boolean = true

In [26]:
"event 1 ok".right.isLeft

res25: Boolean = false

In [27]:
"event 1 ok".right | "something bad"

res26: String = "event 1 ok"

In [28]:
~"event 2 failed!".left[String] | "something good"

res27: String = "event 2 failed!"

In [29]:
"event 1 ok".right map {_ + "!"}

res28: Nothing \/ String = \/-("event 1 ok!")

In [30]:
"event 1 failed!".left ||| "retry event 1 ok".right 

res29: Nothing \/ String = \/-("retry event 1 ok")

## Validation

In [31]:
"event 1 ok".success[String]

res30: Validation[String, String] = Success("event 1 ok")

In [32]:
"event 1 failed!".failure[String]

res31: Validation[String, String] = Failure("event 1 failed!")

In [33]:
("event 1 ok".success[String] |@| "event 2 failed!".failure[String] |@| "event 3 failed!".failure[String]) {_ + _ + _}

res32: Validation[String, String] = Failure("event 2 failed!event 3 failed!")

## NonEmptyList

In [34]:
1.wrapNel

res33: NonEmptyList[Int] = NonEmpty[1]

In [35]:
"event 1 ok".successNel[String]

res34: ValidationNel[String, String] = Success(event 1 ok)

In [36]:
"event 1 failed!".failureNel[String]

res35: ValidationNel[String, String] = Failure(NonEmpty[event 1 failed!])

In [37]:
("event 1 ok".successNel[String] |@| "event 2 failed!".failureNel[String] |@| "event 3 failed!".failureNel[String]) {_ + _ + _}

res36: Validation[NonEmptyList[String], String] = Failure(NonEmpty[event 2 failed!,event 3 failed!])